# json 파일에서 null, null인 부분 제거

In [1]:
import pandas as pd
import numpy as np
import glob
import json
import os
import gc
from tqdm import tqdm

In [2]:
img_list = glob.glob('/data/ephemeral/home/datasets/images/val/*.jpg')
len(img_list)

404

In [11]:
tmp = dict()
with open('/data/ephemeral/home/datasets/jsons/origin_train.json', 'r') as train_json:
    data = json.load(train_json)
    words = data['images']['drp.en_ko.in_house.selectstar_000003.jpg']['words']

    for word_id, word_data in words.items():
        ori = word_data.get('orientation')
        lan = word_data.get('language')
        if ori != None or lan != None:
            tmp[word_id] = word_data

    img_w = data['images']['drp.en_ko.in_house.selectstar_000003.jpg']['img_w']
    img_h = data['images']['drp.en_ko.in_house.selectstar_000003.jpg']['img_h']
    
print(tmp)
print(img_w)
print(img_h)

{'0001': {'points': [[235.83, 91.13], [430.55, 108.13], [433.64, 133.37], [235.83, 119.46]], 'orientation': 'Horizontal', 'language': ['ko']}, '0002': {'points': [[457.34, 113.79], [468.67, 113.79], [468.67, 130.79], [457.34, 130.79]], 'orientation': 'Horizontal', 'language': None}, '0003': {'points': [[493.91, 114.83], [550.43, 119.49], [551.09, 143.67], [493.24, 140.34]], 'orientation': 'Horizontal', 'language': None}, '0004': {'points': [[232.74, 118.95], [345.55, 125.13], [346.58, 149.85], [233.25, 144.7]], 'orientation': 'Horizontal', 'language': ['ko']}, '0005': {'points': [[233.79, 145.49], [370.86, 155.73], [371.15, 178.76], [234.36, 171.65]], 'orientation': 'Horizontal', 'language': None}, '0006': {'points': [[379.68, 156.86], [419.21, 160.2], [419.21, 181.32], [379.68, 181.32]], 'orientation': 'Horizontal', 'language': ['en']}, '0007': {'points': [[233.22, 173.36], [326.21, 177.63], [326.21, 201.8], [233.5, 198.38]], 'orientation': 'Horizontal', 'language': ['ko']}, '0008': {

In [13]:
with open('/data/ephemeral/home/datasets/jsons/val.json', 'r') as f:
    data = json.load(f)

data['images']['drp.en_ko.in_house.selectstar_000003.jpg'] = {"words": tmp}
data['images']['drp.en_ko.in_house.selectstar_000003.jpg']['img_w'] = img_w
data['images']['drp.en_ko.in_house.selectstar_000003.jpg']['img_h'] = img_h

with open('/data/ephemeral/home/datasets/jsons/val.json', 'w') as modified:
    json.dump(data, modified, indent=4)

In [14]:
def val_v2_json():
    # 원본 annotation파일 오픈
    with open('/data/ephemeral/home/datasets/jsons/origin_val.json', 'r') as train_json:
        origin_data = json.load(train_json)
    
    img_path_list = glob.glob('/data/ephemeral/home/datasets/images/val/*.jpg') # img list
    img_path_list.sort() # 확인하기 편하게 이름 순서로 정렬
    tbar = tqdm(img_path_list) 
    tmp_list = list()
    cnt = 0

    for img_path in tbar:
        cnt += 1
        img_name = img_path.split('/')[-1] # 이미지 이름 추출
        words = origin_data['images'][img_name]['words'] # 사진의 words 정보 가져오기
        tmp = dict() # 데이터 임시 저장
        # 모든 word 검사
        for word_id, word_data in words.items():
            ori = word_data.get('orientation')
            lan = word_data.get('language')
            # orientation, language 모두 None인 경우 특수문자로 보인다. -> 일반 문자만 저장할 예정
            if ori != None or lan != None:
                tmp[word_id] = word_data
        
        img_w = origin_data['images'][img_name]['img_w']
        img_h = origin_data['images'][img_name]['img_h']
        tmp_list.append({img_name: {'words': tmp, "img_w": img_w, "img_h": img_h}})

        if cnt % 100 == 0:
            with open('/data/ephemeral/home/datasets/jsons/val.json', 'r') as f:
                data = json.load(f)
            for current in tmp_list:
                key = list(current.keys())[0]
                data['images'][key] = current[key]
            with open('/data/ephemeral/home/datasets/jsons/val.json', 'w') as modified:
                json.dump(data, modified, indent=4)
        
    with open('/data/ephemeral/home/datasets/jsons/val.json', 'r') as f:
        data = json.load(f)
    for current in tmp_list:
        key = list(current.keys())[0]
        data['images'][key] = current[key]
    with open('/data/ephemeral/home/datasets/jsons/val.json', 'w') as modified:
        json.dump(data, modified, indent=4)

In [15]:
val_v2_json()

100%|██████████| 404/404 [00:02<00:00, 154.80it/s]
